In [1]:
import gurobipy as gp
from gurobipy import GRB

# Facility


In [2]:
demand = [15, 18, 14, 20]
capacity = [20, 22, 17, 19, 18]

fixedCosts = [12000, 15000, 17000, 13000, 16000]

transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]]

plants = range(len(capacity))
warehouses = range(len(demand))

In [3]:
m = gp.Model("facility")

# Plant open decision variables: open[p] == 1 if plant p is open.
open = m.addVars(plants,
                 vtype=GRB.BINARY,
                 obj=fixedCosts,
                 name="open")

# Transportation decision variables: transport[w,p] captures the
# optimal quantity to transport to warehouse w from plant p
transport = m.addVars(warehouses, plants, obj=transCosts, name="trans")

m.ModelSense = GRB.MINIMIZE

Restricted license - for non-production use only - expires 2025-11-24


In [4]:
# Production constraints
# Note that the right-hand limit sets the production to zero if the plant
# is closed
m.addConstrs(
    (transport.sum('*', p) <= capacity[p]*open[p] for p in plants), "Capacity")

# Demand constraints
m.addConstrs(
    (transport.sum(w) == demand[w] for w in warehouses),
    "Demand")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [5]:
m.write('facilityPY.lp')

# Guess at the starting point: close the plant with the highest fixed costs;
# open all others

# First open all plants
for p in plants:
    open[p].Start = 1.0

# Now close the plant with the highest fixed cost
print('Initial guess:')
maxFixed = max(fixedCosts)
for p in plants:
    if fixedCosts[p] == maxFixed:
        open[p].Start = 0.0
        print('Closing plant %s' % p)
        break
print('')

# Use barrier to solve root relaxation

times = []

for j in range (6):
    m.Params.Method = j
    time = []
    for i in range(100):
        m.optimize()
        time.append(m.Runtime)
        m.reset()
    times.append(time)

# Solve
#m.optimize()

# Print solution
# print('\nTOTAL COSTS: %g' % m.ObjVal)
# print('SOLUTION:')
# for p in plants:
#     if open[p].X > 0.99:
#         print('Plant %s open' % p)
#         for w in warehouses:
#             if transport[w, p].X > 0:
#                 print('  Transport %g units to warehouse %s' %
#                       (transport[w, p].X, w))
#     else:
#         print('Plant %s closed!' % p)


Initial guess:
Closing plant 2

Set parameter Method to value 0
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 25 columns and 45 nonzeros
Model fingerprint: 0x36b45dc0
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]

User MIP start produced solution with objective 210500 (0.01s)
Loaded user MIP start with objective 210500

Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)

Root relaxation: objective 1.998333e+05, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

In [6]:
import numpy as np
dict_methods = {0: "primal simplex", 1: "dual simplex", 2: "barrier",3: "concurrent", 4: "deterministic concurrent", 5: "deterministic concurrent simplex"}

In [7]:
for i in range(6):
    print(f'for method \"{dict_methods.get(i)}\":\nmean value: {np.mean(np.array(times[i]))}\nvarience: {np.var(np.array(times[i]))}')

for method "primal simplex":
mean value: 0.043183319568634033
varience: 0.00023822672536022652
for method "dual simplex":
mean value: 0.0316648268699646
varience: 0.00013054950208708647
for method "barrier":
mean value: 0.0787766170501709
varience: 0.0004744142354191809
for method "concurrent":
mean value: 0.048547377586364744
varience: 0.00014086399713212358
for method "deterministic concurrent":
mean value: 0.04442797183990479
varience: 0.00012073196809658385
for method "deterministic concurrent simplex":
mean value: 0.044006311893463136
varience: 0.00021481390876871838
